# <center>Đồ án 3: Linear Regression</center>

# Thông tin sinh viên

- Họ và tên: Bùi Quang Thành
- MSSV: 20127329
- Lớp: 20CLC08

# Import

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
np.set_printoptions(suppress=True) # Hiển thị định dạng numy ở số thực gồm các chữ số thập phân sau dấu chấm

# Đọc dữ liệu

In [2]:
# Đọc dữ liệu bằng pandas
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Lấy các đặc trưng X và giá trị mục tiêu y cho các tập huấn luyện (train) và kiểm tra (test)
# ataframe (chứa 10 đặc trưng huấn luyện)
X_train = train.iloc[:,: -1] 
y_train = train.iloc[:, -1]     # Series    (chứa 1 giá trị mục tiêu kiểm tra)

X_test = test.iloc[:, :-1]      # Dataframe (chứa 10 đặc trưng kiểm tra)
y_test = test.iloc[:, -1]       # Series    (chứa 1 giá trị mục tiêu kiểm tra)

headers=['Adult Mortality', 'BMI', 'Polio', 'Diphtheria', 'HIV/AIDS', 'GDP', 'Thinness age 10-19', 'Thinness age 5-9', 'Income composition of resources', 'Schooling', 'Life expectancy']
fiveFold=5

# Cài đặt hàm

In [3]:
def toNumpy(X_train,Y_train,X_test,Y_test): # chuyển đổi dữ liệu về numpy
    X_train=X_train.to_numpy()
    Y_train=Y_train.to_numpy()
    X_test=X_test.to_numpy()
    Y_test=Y_test.to_numpy()
    return X_train,Y_train,X_test,Y_test

In [4]:
X_train,y_train,X_test,y_test=toNumpy(X_train,y_train,X_test,y_test)

In [5]:
def rmse(y, y_hat): # hàm tính rmse
    return math.sqrt(np.mean((y.ravel() - y_hat.ravel())**2))

In [6]:
def caculate_Coefficients_Matrix(X_train,Y_train):  # tính ma trận hệ số
    X_pinv = np.linalg.inv(X_train.T @ X_train) @ X_train.T    # np.linalg.pinv(X)
    Coefficients = X_pinv @ Y_train
    return Coefficients

In [7]:
def predict(Coefficients,X_test): # hàm dự đoán hồi quy tuyến tính 
    if len(X_test)==10:
          result=np.sum(Coefficients.ravel() * X_test, axis=0)
    else: 
     result=np.sum(Coefficients.ravel() * X_test, axis=1) 
    return result

In [8]:
def rankingFeature(distances,headers,bestIndexOrders): # Vẽ bảng xếp hạng từng đặc trưng
    x_hat_data = []
    rank=1
    for i in range(10):
        row = []
        row.append('x'+ str(i + 1))
        row.append(headers[bestIndexOrders[i]])
        row.append(np.mean(distances[bestIndexOrders[i]]))
        row.append(rank)
        rank=rank+1
        x_hat_data.append(row)
    
    return pd.DataFrame(x_hat_data, columns=['x', 'Tính chất','RMSE','Xếp hạng'])

In [9]:
def takeNFeatureBestIndex(data,numberFeature): # hàm lấy ra index phần tử có rmse trung bình từ nhỏ nhất -> lớn nhất
    temp=data.copy()
    temp = np.sort(temp)
    temp = temp[0:numberFeature]
    indexs=[]
    for i in range(0,len(data)):
        for j in range(0,len(temp)):
            if temp[i]==data[j]:
             indexs.append(j)
             
            
   
    return indexs

In [10]:
def inforDesignModel(rmses): # vẽ bảng thông tin của 4 mô hình tự xây dựng
    x_hat=[]
    header=["Sử dụng 2 đặc trưng tốt nhất ","Sử dụng đặc trưng được chuẩn hóa(mũ 0.4)","Đặc trưng tốt nhất + tốt nhì","Sử dụng 10 đặc trưng + bias"]
    count=1
    for i in range(len(rmses)):
        row=[]
        row.append(count)
        count=count+1
        row.append(header[i])
        row.append(rmses[i])
        x_hat.append(row)
    return pd.DataFrame(x_hat, columns=['STT', 'Mô hình','RMSE'])

# Yêu cầu 1a: Sử dụng toàn bộ 10 đặc trưng đề bài cung cấp (2 điểm) 

In [11]:
Coefficients=caculate_Coefficients_Matrix(X_train,y_train)

result=predict(Coefficients,X_test)


In [12]:
print("Mô hình y = {}*x".format(Coefficients))
print("RMSE toàn bộ 10 đặc trưng:",rmse(y_test.reshape(len(y_test),),result))

Mô hình y = [ 0.01510136  0.09021998  0.04292182  0.13928912 -0.56733283 -0.00010077
  0.74071344  0.1909358  24.50597359  2.39351661]*x
RMSE toàn bộ 10 đặc trưng: 7.064046430584037


Công thức hồi quy

$$\text{Life expectancy} = a1x1+a2x2+a3x3+a4x4+a5x5+a6x6+a7x7+a8x8+a9x9+a10x10$$


In [13]:
print("Coefficients",Coefficients)

Coefficients [ 0.01510136  0.09021998  0.04292182  0.13928912 -0.56733283 -0.00010077
  0.74071344  0.1909358  24.50597359  2.39351661]


Trong đó

| xi      | ai |
| ----------- | ----------- |
|x1=Adult Mortality      |   a1=Coefficients[0]     |
| x2=BMI   |a2=Coefficients[1]    |
|x3=Polio     | a3=Coefficients[2]  |
|   x4=Diphtheria        | a4=Coefficients[3] |
| x5=HIV/AIDS      | a5=Coefficients[4]        |
| x6=GDP  | a6=Coefficients[5]         |
| x7=Thinness age 10-19     | a7=Coefficients[6]  |
| x8=Thinness age 5-9 | a8=Coefficients[7]  |
| x9=Income composition of resources     | a9=Coefficients[8]        |
| x10=Schooling  | a10=Coefficients[9]         |


# Yêu cầu 1b: Xây dựng mô hình sử dụng duy nhất 1 đặc trưng, tìm mô hình cho kết quả tốt nhất (2 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ đặc trưng

In [14]:
def fiveFoldCrossValidation(train_test_split,test_data,n_splits,message): #thêm tham số message bởi vì mô hình xây dựng ở câu c cần có ma trận hệ số 
       
        split_train = train_test_split[:] # sao chép dữ liệu
        test_train=test_data[:]# sao chép dữ liệu
       
        n = int(np.round(len(split_train) /n_splits))
      
        models_train = [split_train[i*n:i*n+n, :] for i in range(n_splits-1)]  # lấy dữ liệu từ 0-> n
        models_train.append(split_train[(n_splits-1)*n:, :]) # lấy dữ liệu từ n->hết
        distance=[]
        models_test = [test_train[i*n:i*n+n] for i in range(n_splits-1)]
        models_test.append(test_train[(n_splits-1)*n:,])

      
        for i in range(0,n_splits):
            
            A_train_data = np.concatenate(models_train[:-1], axis=0)
            A_test_data = models_train[-1]
            B_train_data= np.concatenate(models_test[:-1], axis=0)
            B_test_data = models_test[-1]
            x=caculate_Coefficients_Matrix(A_train_data,B_train_data) # tính toán ma trận hệ số dựa trên các dữ liệu huấn luyện
           
            if(message=="Have bias"): # Sử dụng cho mô hình có kết hợp bias
               
                predict=np.sum(x[:-1].ravel()*A_test_data[:,:-1], axis=1) 
                predict=predict+x[-1]
                
                
            else:
                
                predict=np.sum(x.ravel() * A_test_data, axis=1) 

            distance.append(rmse(B_test_data,predict))
            # distance.append(np.mean(np.absolute(B_test_data - (np.dot(A_test_data, x)))))
            # Bắt đầu hoán đổi dữ liệu tập huấn luyện
            tempA = models_train[-1]
            models_train[-1] = models_train[i]
            models_train[i] = tempA

            # Bắt đầu hoán đổi dữ liệu tập kiểm tra
            tempB = models_test[-1]
            models_test[-1] = models_test[i]
            models_test[i] = tempB
            
        return (np.mean(np.array(distance))),distance

In [15]:
def bestNFeaturedIndex(data,numberFeature): # hàm tính toán rmse trung bình của 10 đặc trưng 
    Means=[]
    distances=[]
    for i in range(len(data[1])):
        
        mean,distance=fiveFoldCrossValidation(X_train[:, i:i + 1],y_train,fiveFold,"No bias") 
        Means.append(mean)
        distances.append(distance)
    
    
    return takeNFeatureBestIndex(np.array(Means),numberFeature),distances

In [16]:
# [9, 8, 3, 2, 1, 0, 7, 6, 5, 4] -->(kết quả sử dụng phương pháp cross validation để tìm đặc trưng tốt nhất) , được sắp xếp từ theo thứ tự từ nhỏ --> lớn
bestIndexOrders,distances = bestNFeaturedIndex(X_train,10)
best_feature_model=caculate_Coefficients_Matrix(X_train[:,bestIndexOrders[0]:bestIndexOrders[0]+1],y_train)

result=predict(best_feature_model,X_test[:,bestIndexOrders[0]:bestIndexOrders[0]+1])


In [17]:
print("Mô hình y = {}*x".format(best_feature_model))
print("RMSE của 1 đặc trưng tốt nhất :", rmse(y_test.reshape(len(y_test),),result))


Mô hình y = [5.5573994]*x
RMSE của 1 đặc trưng tốt nhất : 10.26095039165537


In [18]:
rankingFeature(distances,headers,bestIndexOrders) # bảng xếp hạng 

,x,Tính chất,RMSE,Xếp hạng
0,x1,Schooling,11.820071,1
1,x2,Income composition of resources,13.299791,2
2,x3,Diphtheria,16.019288,3
3,x4,Polio,17.912636,4
4,x5,BMI,27.963793,5
5,x6,Adult Mortality,46.767300,6
6,x7,Thinness age 5-9,51.775059,7
7,x8,Thinness age 10-19,51.899815,8
8,x9,GDP,60.450393,9
9,x10,HIV/AIDS,69.081327,10


Công thức hồi quy

$$\text{Life expectancy} =a0*Schooling$$

In [19]:
print("Coefficients",best_feature_model)

Coefficients [5.5573994]


| xi      | ai |
| -------      | ------ |
| x0=Schooling  | a0=Coefficients[0]         |

# Yêu cầu 1c: Sinh viên tự xây dựng mô hình, tìm mô hình cho kết quả tốt nhất (3 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ $m$ mô hình mà sinh viên thiết kế



$$\text{Các mô hình được xây dựng}$$
##### Features
- Mô hình **1** : là sự kết hợp của 2 đặc trưng tốt nhất sau khi sử dụng phương pháp  **5-fold Cross Validation** 

- Mô hình **2** : là sự kết hợp của 3 đặc trưng tốt nhất và đã được chuẩn hóa (chi tiết phía dưới) sau khi sử dụng phương pháp  **5-fold Cross Validation** 
- Mô hình **3** : là 1 **"đặc trưng mới"** được cộng từ 2 đặc trưng tốt nhất sau khi sử dụng phương pháp  **5-fold Cross Validation** 
- Mô hình **4** : là sự kết hợp của 10 đặc trưng và độ lệch (bias) 


##### Mô Hình 1

In [20]:
def model2FeatureBestData(data,bestIndexOrders): #Sử dụng 2 đặc trưng tốt nhất. 
    return data[:,[bestIndexOrders[0],bestIndexOrders[1]]] # trả về 1 tập dữ liệu gồm 2 đặc trưng tốt nhất 

In [21]:

def rmse_Model2FeatureBest(data,bestIndexOrders,y_train): 
    X_train_2Model=model2FeatureBestData(data,bestIndexOrders)
    Coefficients=caculate_Coefficients_Matrix(X_train_2Model,y_train)
    mean,_=fiveFoldCrossValidation(X_train_2Model,y_train,fiveFold,"No bias") # trả về giá trị trung bình sau khi thực hiện Cross Validation
    return mean,Coefficients

##### Mô Hình 2

In [22]:
def model_3Feature_Best_Standardized_Data(data,bestIndexOrders): # chuan hoa mo hinh 
    return data[:,[bestIndexOrders[0],bestIndexOrders[1],bestIndexOrders[2]]]**0.4

In [23]:
def rmse_Model3FeatureBestPower(data,bestIndexOrders,y_train):
    X_train_3Model=model2FeatureBestData(data,bestIndexOrders)
    Coefficients=caculate_Coefficients_Matrix(X_train_3Model,y_train)
    X_train_3Model=model_3Feature_Best_Standardized_Data(data,bestIndexOrders)
    mean,_=fiveFoldCrossValidation(X_train_3Model**0.4,y_train,fiveFold,"No bias")
    return mean,Coefficients

##### Mô Hình 3

In [24]:
def model_2Feature_Best_Plus_Data(data,bestIndexOrders): # mô hình cộng 2 dặc trưng
    return (data[:,[bestIndexOrders[0]]]+data[:,[bestIndexOrders[1]]])


In [25]:
def rmse_Model2FeatureBestPlus(data,bestIndexOrders,y_train):# rmse mô hình cộng 2 dặc trưng
    X_train_2ModelPlus=model_2Feature_Best_Plus_Data(data,bestIndexOrders)
    Coefficients=caculate_Coefficients_Matrix(X_train_2ModelPlus,y_train)
    mean,_=fiveFoldCrossValidation(X_train_2ModelPlus,y_train,fiveFold,"No bias")
    return mean,Coefficients

##### Mô Hình 4

In [26]:
def model_Matrix_Algebra_HaveBias(data):#Hàm xây dựng dựa trên 10 đặc trưng và độ lệch (bias)
    temp_data=data.copy()
    newComlumn=np.ones((len(temp_data),1)) # khởi tạo 1 cột mới chứa toàn số 0
    temp_data=np.append(temp_data,newComlumn,axis=1) # thêm cột mới tạo vào dữ liệu có sẵn
    coefficients =caculate_Coefficients_Matrix(temp_data,y_train) # tính ma trận hệ số 
    
    bias=coefficients[-1] # độ lệch
    return coefficients[:-1],bias
    

In [27]:
def rmse_Model_Matrix_Algebra_HaveBias(X_train,y_train):
    temp_data=X_train.copy()
    newComlumn=np.ones((len(temp_data),1)) # khởi tạo 1 cột mới chứa toàn số 0
    temp_data=np.append(temp_data,newComlumn,axis=1) # thêm cột mới tạo vào dữ liệu có sẵn
    mean,_=fiveFoldCrossValidation(temp_data,y_train,fiveFold,"Have bias")
    coefficients,bias=model_Matrix_Algebra_HaveBias(X_train)
    
    return mean,coefficients,bias

##### Mô hình tốt nhất

In [28]:
def my_best_model(data,bestIndexOrders,y_train):# hàm lấy ra mô hình có rmse nhỏ nhất 
    rmses=[]
    rmseModel1,model1=rmse_Model2FeatureBest(X_train,bestIndexOrders,y_train)
    rmses.append(rmseModel1)
    rmseModel2,model2=rmse_Model3FeatureBestPower(X_train,bestIndexOrders,y_train)
    rmses.append(rmseModel2)
    rmseModel3,model3=rmse_Model2FeatureBestPlus(X_train,bestIndexOrders,y_train)
    rmses.append(rmseModel3)
    rmseModel4,model4,bias=rmse_Model_Matrix_Algebra_HaveBias(X_train,y_train)
    rmses.append(rmseModel4)
    return rmses,model4,bias


In [29]:
rmses,model4,bias=my_best_model(X_train,bestIndexOrders,y_train)



predict=np.sum(model4.ravel() * X_test, axis=1) 
predict=predict+bias

print("Mô hình y = {}*x + {}".format(model4,bias))
print("RMSE của mô hình tự xây dựng ",rmse(y_test,predict))

Mô hình y = [-0.0166735   0.04572407  0.00145796  0.02528803 -0.49803865  0.00006294
 -0.04033714 -0.05648104 12.70589597  0.70769451]*x + 52.43340343662611
RMSE của mô hình tự xây dựng  3.4610550812967555


In [30]:
inforDesignModel(rmses)

,STT,Mô hình,RMSE
0,1,Sử dụng 2 đặc trưng tốt nhất,11.431486
1,2,Sử dụng đặc trưng được chuẩn hóa(mũ 0.4),7.129564
2,3,Đặc trưng tốt nhất + tốt nhì,11.663742
3,4,Sử dụng 10 đặc trưng + bias,4.043032


Công thức hồi quy

$$\text{Life expectancy} = a1x1+a2x2+a3x3+a4x4+a5x5+a6x6+a7x7+a8x8+a9x9+a10x10 +bias$$

In [31]:
print("Coefficients: ",model4)
print("Coefficients[10] = Bias: ",bias)

Coefficients:  [-0.0166735   0.04572407  0.00145796  0.02528803 -0.49803865  0.00006294
 -0.04033714 -0.05648104 12.70589597  0.70769451]
Coefficients[10] = Bias:  52.43340343662611


Trong đó

| xi      | ai |
| ----------- | ----------- |
|x1=Adult Mortality      |   a1=Coefficients[0]     |
| x2=BMI   |a2=Coefficients[1]    |
|x3=Polio     | a3=Coefficients[2]  |
|   x4=Diphtheria        | a4=Coefficients[3] |
| x5=HIV/AIDS      | a5=Coefficients[4]        |
| x6=GDP  | a6=Coefficients[5]         |
| x7=Thinness age 10-19     | a7=Coefficients[6]  |
| x8=Thinness age 5-9 | a8=Coefficients[7]  |
| x9=Income composition of resources     | a9=Coefficients[8]        |
| x10=Schooling  | a10=Coefficients[9]         |
| x11=1 | a11=Coefficients[10]         |
